# Plot motion

In [1]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets ModelSupport TextModels

Installing packages:
	.package(path: "/notebooks/language2motion.gt")
		Datasets
		ModelSupport
		TextModels
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpvqieush7/swift-install
[1/2] Compiling Datasets ArrayUtils.swift
[2/3] Compiling TextModels Attention.swift
/notebooks/language2motion.gt/Sources/Models/Text/BERT.swift:776:32: warning: 'TensorFlowCheckpointReader' is deprecated: TensorFlowCheckpointReader will be removed in S4TF v0.11. Please use CheckpointReader from swift-models
(https://github.com/tensorflow/swift-models/blob/master/Support/Checkpoints/CheckpointReader.swift)
instead.
        let checkpointReader = TensorFlowCheckpointReader(checkpointPath: fileURL.path)
                               ^
[3/4] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[4/4] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [2]:
import TensorFlow
import Foundation
import Datasets
import ModelSupport
import TextModels
import PythonKit

In [11]:
// %include "EnableIPythonDisplay.swift"
let plt = Python.import("matplotlib.pyplot")
// IPythonDisplay.shell.enable_matplotlib("inline")

## load dataset

In [3]:
let datasetSize: DatasetSize = .mini

let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
// motion_dataset2.10Hz.39728.plist
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset_v3.10Hz.\(datasetSize.rawValue)plist")
let langDatasetURL = dataURL.appendingPathComponent("labels_ds_v2.csv")

In [4]:
let batchSize = 4
let maxTextSequenceLength =  20
let maxMotionLength =  100

In [5]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor2(vocabulary: vocabulary, tokenizer: tokenizer, maxTextSequenceLength: maxTextSequenceLength, maxMotionLength: maxMotionLength)

In [6]:
/// load dataset
print("\nLoading dataset...")

var dataset = try Lang2Motion(
    motionDatasetURL: motionDatasetURL,
    langDatasetURL: langDatasetURL,
    batchSize: batchSize
) { (example: Lang2Motion.Example) -> LangMotionBatch in    
    let singleBatch = textProcessor.preprocess(example: example)
    return singleBatch
}

print("Dataset acquired.")


Loading dataset...
MotionDataset2(motionSamples: 1030)
keeping 834 annotatated motions
keeping 834 longer motions, with minimum 10 frames
Scaling motions...
Motions scaled.
Dataset acquired.


## plot motion sample

In [7]:
let ms = dataset.motionDataset.motionSamples[0]

In [12]:
extension Tensor where Scalar: NumpyScalarCompatible, Scalar: Numeric {
    public func motionToImg(url: URL, padTo: Int = 500, descr: String) {
        let motion = self.paddedAndCropped(to: padTo).motion
        let x = plt.subplots()
        let ax = x[1]
        // cmaps: viridis, gist_rainbow, bwr, seismic, coolwarm, hsv, plasma*, PRGn, twilight_shifted, Spectral...
        ax.imshow(motion.makeNumpyArray().T, extent: [0, padTo, 0, motion.shape[1]], cmap: "Spectral")
        ax.set_title("\(descr)")
        plt.savefig(url.path)
    }
}

In [13]:
ms.motion.motionToImg2(url: dataURL.appendingPathComponent("motion_images/foo4.png"), padTo: maxMotionLength, descr: "\(ms.description) \(ms.annotations[0])")

In [41]:
extension Tensor where Scalar: NumpyScalarCompatible, Scalar: Numeric {
    public func paddedTo(padTo: Int = 500) -> Tensor {
        let rank = self.shape.count
        let currentWidth = self.shape[0]
        let paddingSize = Swift.max(padTo - currentWidth, 0)
        print("self.shape: \(self.shape)")
        var sizes: [(before: Int, after: Int)] = [(before: 0, after: paddingSize)]
        if rank > 1 {
            sizes.append((before: 0, after: 0))
        }        
        let tensor = self.padded(forSizes: sizes, with: 0)
        return tensor
    }
}

public func motionToImg4(url: URL, motion: Tensor<Float>, motionFlag: Tensor<Int32>, padTo: Int = 500, descr: String = "") {
    let currentWidth = motion.shape[0]
    let paddingSize = Swift.max(padTo - currentWidth, 0)
    let motion = motion.paddedTo(padTo: padTo)
    let motionFlag = motionFlag.paddedTo(padTo: padTo)
    print("motion.shape: \(motion.shape)")
    print("motionFlag.shape: \(motionFlag.shape)")
    print("max: \(motion.max())")
    let motionFlag2 = Tensor<Float>(motionFlag).expandingShape(at: 1)*motion.max()
    let joined = Tensor(concatenating: [motionFlag2, motion], alongAxis: 1)
    print("joined.shape: \(joined.shape)")
    
    let x = plt.subplots()
    let ax = x[1]
    // cmaps: viridis, gist_rainbow, bwr, seismic, coolwarm, hsv, plasma*, PRGn, twilight_shifted, Spectral...
    ax.imshow(joined.makeNumpyArray().T, extent: [0, padTo, 0, joined.shape[1]], cmap: "Spectral")
    ax.set_title("\(descr)")
    plt.savefig(url.path)
}

In [42]:
var motionFlag = Tensor<Int32>(repeating: 1, shape: [55])
motionToImg4(url: dataURL.appendingPathComponent("motion_images/foo7.png"), 
             motion: ms.motion, motionFlag: motionFlag, padTo:maxMotionLength)

self.shape: [61, 47]
self.shape: [55]
motion.shape: [100, 47]
motionFlag.shape: [100]
max: 2.55286
joined.shape: [100, 48]
